In [8]:
import ionpy
from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
import pandas as pd

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext autoreload
%load_ext yamlmagic
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [3]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
df

,phase,epoch,loss,dice_score,data,annotator,axis,dataset,slice_batch_size,slicing,...,convs_per_block,filters,in_channels,out_channels,optim,lr,epochs,eval_freq,path,augmentations
0,train,0,0.983604,0.008578,ese.experiment.datasets.WMH,observer_o12,0,WMH,1,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,NaN
1,val,0,0.977503,0.011451,ese.experiment.datasets.WMH,observer_o12,0,WMH,1,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,NaN
2,train,1,0.980786,0.032685,ese.experiment.datasets.WMH,observer_o12,0,WMH,1,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,NaN
3,train,2,0.972140,0.067008,ese.experiment.datasets.WMH,observer_o12,0,WMH,1,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,NaN
4,train,3,0.848856,0.115145,ese.experiment.datasets.WMH,observer_o12,0,WMH,1,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2832,train,496,0.344877,0.546215,ese.experiment.datasets.WMH,observer_o12,0,WMH,8,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,"[{""RandomAffine"": {""degrees"": [0, 360], ""p"": 0..."
2833,train,497,0.354526,0.537993,ese.experiment.datasets.WMH,observer_o12,0,WMH,8,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,"[{""RandomAffine"": {""degrees"": [0, 360], ""p"": 0..."
2834,train,498,0.337487,0.553440,ese.experiment.datasets.WMH,observer_o12,0,WMH,8,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,"[{""RandomAffine"": {""degrees"": [0, 360], ""p"": 0..."
2835,train,499,0.349431,0.533210,ese.experiment.datasets.WMH,observer_o12,0,WMH,8,dense,...,2,"(64, 64, 64, 64)",1,1,torch.optim.Adam,0.0003,500,50,/storage/vbutoi/scratch/ESE/WMH_aug_runs/20230...,"[{""RandomAffine"": {""degrees"": [0, 360], ""p"": 0..."


In [6]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


In [ ]:
%%yaml dataset_cfg 

data:
  _class: ese.experiment.datasets.WMH
  annotator: observer_o12
  axis: 0
  dataset: WMH  
  split: val
  slice_batch_size: 1
  task: Amsterdam 
  version: 0.2

In [2]:
def get_dataset_perf(exp, split, dataset_config):

    dataset_cls = absolute_import(dataset_config.pop("_class"))
    dataset = data_cfg.pop("dataset")
    Dataset = dataset_cls(dataset=dataset, split=split, **dataset_config)
    dataloader = DataLoader(Dataset, shuffle=False, drop_last=False)

    items = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            x, y = to_device(batch, exp.device)
            yhat = exp.model(x)
            loss = exp.loss_func(yhat, y)
            items.append({
                "x": x.cpu().numpy(),
                "y": y.cpu().numpy(),
                "yhat": yhat.detach().cpu().numpy(),
                "loss": loss.item()
            })
    return items

In [3]:
from ionpy.util import Config
#from ese.experiment.analysis import get_dataset_perf

# val_perf is a dict where each item is the subj id
# with the y, ypred, yloss, ydice
val_perf = get_dataset_perf(
    exp=best_exp, 
    spli="val",
    dataset_config=Config(dataset_config)
    )

ModuleNotFoundError: No module named 'torch'